Nama Anggota Kelompok E:
- Aulia Nisrina Rosanita / 2206051380
- Angelia Huwinata / 2206051595
- Renata Shaula Alfino Ritonga / 2206815812
- Amira Shohifa / 2206829130
- Zavier Raiyan Dana / 2206829875

## Import Packages

In [1]:
#Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy as sp
from scipy import stats
from math import fsum, sqrt
from timeit import default_timer as timer

import matplotlib.gridspec as gridspec
from pandas.api.types import CategoricalDtype
%matplotlib inline

## Import Dataset

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
link = 'https://drive.google.com/file/d/16OtLnKSv5z9mD-yE7EPfS09C1oUPGmXJ/view'

id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('data_kesehatan-Mental-SRQ-2023--Filtered.csv')

data = pd.read_csv('data_kesehatan-Mental-SRQ-2023--Filtered.csv')

<ipython-input-3-fcdcd190a0da>:8: DtypeWarning: Columns (6,77) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_kesehatan-Mental-SRQ-2023--Filtered.csv')


In [4]:
ord_umur = CategoricalDtype(categories=['15 - 24','25 - 34', '35 - 44', '45 - 54', '55 - 64', '≥ 65'], ordered=True)
ord_IMT = CategoricalDtype(categories=['Kurus', 'Sangat Kurus', 'Normal', 'Gemuk', 'Obesitas'], ordered=True)
data['Kelompok Umur'] = data['Kelompok Umur'].astype(ord_umur)
data['IMT'] = data['IMT'].astype(ord_IMT)
data['Kelompok Umur'] = data['Kelompok Umur'].cat.codes
data['IMT'] = data['IMT'].cat.codes

## K-Means dengan Parallel Programming dan perbedaannya dengan Sequential K-Means

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
from timeit import default_timer as timer
import matplotlib.pyplot as plt

# Randomly generating initial centroids
def choose_centroids(n,k):
    import random as rd
    centroids_idx = rd.sample(range(n),k)
    centroids_idx.sort()
    return centroids_idx

def distance(centroid,datapoint):
    return sqrt(np.square(centroid-datapoint).sum())

def Kmeans(df,cidx,n,k):
    iterations = 0
    time = []
    centroids = df.loc[cidx]
    clusters = np.zeros(n,dtype=int)

    while iterations < 100:
        startit = timer()

        iterations += 1
        old_clusters = clusters.copy()

        #Part to be Parallelized
        for i in range(n):
            dist = []
            for j in range(k):
                dist.append(distance(centroids.loc[centroids_idx[j]],df.loc[i]))
            clusters[i] = dist.index(min(dist))
        #Parallellizing ends here
        endit = timer() - startit
        time.append(endit)

        if np.array_equal(clusters,old_clusters):
            break

        # update Cluster centroids
        for j in range(k):
            centroids.loc[centroids_idx[j]] = df.loc[clusters == j].mean(axis=0)

    return np.mean(np.asarray(time)),clusters

import multiprocessing as mp
from scipy.spatial.distance import cdist
from itertools import repeat

def find_cluster(data,centroids):
    distances = cdist(data,centroids,'euclidean')
    return np.argmin(distances, axis = 1)

def mpKmeans(df,cidx,n,k,cpus):
    iterations = 0
    time = []
    centroids = df[cidx]
    clusters = np.zeros(n,dtype=int)

    split_data = np.array_split(df, cpus)

    while iterations < 100:
        startit = timer()

        iterations += 1
        old_clusters = clusters.copy()

        arg = zip(split_data, repeat(centroids))

        pool = mp.Pool(processes=cpus)
        split_clusters = pool.starmap(find_cluster,arg)
        pool.close()
        pool.join()

        clusters = np.concatenate(split_clusters)

        endit = timer() - startit
        time.append(endit)

        if np.array_equal(clusters,old_clusters):
            break

        # update Cluster centroids
        for j in range(k):
            centroids[j] = df[clusters == j].mean(axis=0)

    return np.mean(np.asarray(time)),clusters

if __name__ == "__main__":
    # df = pd.read_csv('data_kesehatan-Mental-SRQ-2023--Filtered.csv', sep=',')
    df = data[[ 'Kelompok Umur', 'IMT']]
    # df = pd.read_csv('data\\winequality-white.csv', sep=';')
    # df = df[df.columns[:8]]

    # Parameters for Kmeans
    k = [3, 4]  # Number of Clusters
    n = df.shape[0]  # Number of datapoints
    att = df.shape[1]  # Number of Attributes

    print("Number of Datapoints =", n)
    print("Number of Attributes =", att)

    # time taken to run different algorithms
    time_kmeans = np.zeros(len(k), dtype=float)
    time_mp = np.zeros(len(k), dtype=float)

    # iteration count in different algorithms
    iter_kmeans = np.zeros(len(k), dtype=float)
    iter_mp = np.zeros(len(k), dtype=float)

    # Differences in the results
    diff_mp_km = np.zeros(len(k), dtype=int)

    # Initial centroids
    for i in range(len(k)):
        centroids_idx = choose_centroids(n, k[i])
        print("Number of Clusters to be formed:", k[i])
        print("\tTime\tIterations")

        # running normal kmeans algorithms
        start_k = timer()
        iter_kmeans[i], clusters_kmeans = Kmeans(df, centroids_idx, n, k[i])
        time_kmeans[i] = timer() - start_k
        print('Kmeans:\t%4.4f\t%4.4f' % (time_kmeans[i], iter_kmeans[i]))

        # running kmeans algorithm with parallel computing using Multiprocessing module
        start_m = timer()
        cpus = mp.cpu_count()
        iter_mp[i], clusters_mp = mpKmeans(df.values, centroids_idx, n, k[i], cpus)
        time_mp[i] = timer() - start_m
        print('MProcs:\t%4.4f\t%4.4f' % (time_mp[i], iter_mp[i]))

        # Calculating difference between results
        diff_mp_km[i] = sum(clusters_kmeans != clusters_mp)

        if diff_mp_km[i] != 0:
            print("Cluster Differences: Kmeans Vs Multiprocessing :", diff_mp_km[i])

    # Plotting Total Execution Time
    plt.plot(k, time_kmeans, 'r-', k, time_mp, 'g-')
    plt.title("Perbandingan waktu untuk data berukuran 289801 x 2")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Time Taken to Run Clustering Algorithm")
    plt.legend(("Sequential K-Means", "Parallel K-Means"), loc='best')
    plt.show()

    # Plotting Mean Iteration Time
    plt.plot(k, iter_kmeans, 'r-', k, iter_mp, 'g-')
    plt.title("Perbandingan rata-rata waktu untuk menjalankan 1 iterasi Kmeans clustering")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Mean Time Taken to Run 1 Iteration of KMeans Clustering Algorithm")
    plt.legend(("Sequential K-Means", "Parallel K-Means"), loc='best')
    plt.show()


Number of Datapoints = 289801
Number of Attributes = 2
Number of Clusters to be formed: 3
	Time	Iterations
Kmeans:	1186.0813	395.3472
MProcs:	0.9042	0.1989
Cluster Differences: Kmeans Vs Multiprocessing : 88533
Number of Clusters to be formed: 4
	Time	Iterations


Untuk data berukuran 289801 x 2, berikut perbedaan Algoritma K-means dengan parallel programming dan tanpa parallel programming:

* terlihat untuk k = 3 waktu iterasi yang dibutuhkan pada  kedua algoritma memiliki perbedaan yang cukup jauh. Untuk Kmeans dengan parallel memerlukan waktu 0.5 detik sedangkan k-means tanpa parallel memerlukan waktu 1 menit lebih untuk dijalankan. Namun terdapat perbedaan cluster sebanyak 169884 baris
* untuk k = 4 waktu iterasi yang dibutuhkan pada kedua algoritma memiliki perbedaan yang cukup jauh. Untuk Kmeans dengan parallel memerlukan waktu 0.64 detik sedangkan k-means tanpa parallel memerlukan waktu 1 menit lebih untuk dijalankan. Serta terdapat perbedaan cluster sebanyak 135846 baris.
Dapat disimpulkan bahwa parallel programming membantu mempercepat waktu untuk menjalankan Algoritma kmeans pada data berukuran besar.